Title: Sentiment Analysis on a Custom Text Dataset    
Author: Raikibul Hasan

**Table of contents**<a id='toc0_'></a>    
- [Preparation](#toc1_)    
    - [Install libraries](#toc1_1_1_)    
    - [Import libraries](#toc1_1_2_)    
- [Data Expolaration](#toc2_)  
    - [Load the dataset & overview of the data](#_)     
    - [Identify and handling missing data](#_)      
    - [Visualize the distribution](#_)   
- [Sentiment Analysis](#toc3_)
    